In [42]:
from dotenv import load_dotenv

load_dotenv()

True

In [43]:
from ReAct.react_agent import ReAct_agent, REACT_SYSTEM_PROMPT
from Multi_Agent.mindsearch import MindSearch, PLANNER_SYSTEM_PROMPT
from CQ.cq_solver import CQ_Solver, CQ_SYSTEM_PROMPT

In [ ]:
import json
import os

test_dataset_500 = 'test_dataset_500.json'
if not os.path.exists(test_dataset_500):
    print(f"找不到示範問題檔案: {test_dataset_500}。請先運行之前的程式碼生成該檔案。")
    exit()

with open(test_dataset_500, 'r', encoding='utf-8') as f:
    demo_questions = json.load(f)

# 初始化不同的 Agent 系統和模型


results = []
output_file = 'experiment_results.jsonl'  # 使用 .jsonl 方便逐行寫入 JSON 物件

max_questions = 500

# 模擬運行 10 個問題
for i, question in enumerate(demo_questions[114:max_questions], start=114):
    question_id = f"question_{i+1:03d}"  # 使用 3 位數補零，如 question_115
    print(f"\n--- 處理問題: {question_id} ---")

    # react_gpt4o = ReAct_agent(
    #     llm="gpt-4o", system_prompt=REACT_SYSTEM_PROMPT, max_turns=9)
    # react_llama = ReAct_agent(llm="llama", system_prompt=REACT_SYSTEM_PROMPT, max_turns=9)

    # mindsearch_gpt4o = MindSearch(
    #     llm="gpt-4o", system_prompt=PLANNER_SYSTEM_PROMPT, max_turns=9)
    # mindsearch_llama = MindSearch(llm="llama", system_prompt=PLANNER_SYSTEM_PROMPT)

    cqsolver_gpt4o = CQ_Solver(
        llm="gpt-4o", system_prompt=CQ_SYSTEM_PROMPT, max_turns=9)
    # cqsolver_llama = CQ_Solver(llm="llama", system_prompt=CQ_SYSTEM_PROMPT)

    # systems = {
    #     "ReAct": {"gpt-4o": react_gpt4o, "llama": react_llama},
    #     "MindSearch": {"gpt-4o": mindsearch_gpt4o, "llama": mindsearch_llama},
    #     "CQ_Solver": {"gpt-4o": cqsolver_gpt4o, "llama": cqsolver_llama},
    # }

    systems = {
        # "ReAct": {"gpt-4o": react_gpt4o},
        # "MindSearch": {"gpt-4o": mindsearch_gpt4o},
        "CQ_Solver": {"gpt-4o": cqsolver_gpt4o}
    }

    for system_name, models in systems.items():
        for model_name, agent in models.items():
            print(f"系統: {system_name}, 模型: {model_name}")
            try:
                # 假設每個 Agent 都有一個類似的 run 方法來處理問題
                if hasattr(agent, 'run'):
                    response = agent.run(question)
                    print(f"  回答: {response}")
                    result = {
                        "system": system_name,
                        "model": model_name,
                        "question_id": question_id,
                        "question": question,
                        "answer": response,
                        "error": None
                    }
                else:
                    error_message = f"Agent {system_name} with model {model_name} 沒有 'run' 方法。"
                    print(f"  錯誤: {error_message}")
                    result = {
                        "system": system_name,
                        "model": model_name,
                        "question_id": question_id,
                        "question": question,
                        "answer": None,
                        "error": error_message
                    }
            except Exception as e:
                error_message = str(e)
                print(f"  發生錯誤: {error_message}")
                result = {
                    "system": system_name,
                    "model": model_name,
                    "question_id": question_id,
                    "question": question,
                    "answer": None,
                    "error": error_message
                }

            # 即時保存結果
            with open(output_file, 'a', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False)
                f.write('\n')

print(f"\n實驗已完成。結果已保存至 {output_file}")

In [31]:
import json
import os

# === 檔案路徑 ===
experiment_input = 'experiment_results.jsonl'
output_file = 'experiment_results_2.jsonl'

specified_question_id = ['question_111', 'question_110', 'question_136', 'question_255', 'question_257', 'question_298', 'question_307', 'question_317', 'question_347', 'question_433', 'question_442', 'question_445', 'question_476', 'question_480', 'question_481', 'question_492', 'question_397', 'question_414', 'question_423', 'question_436', 'question_444', 'question_450', 'question_451', 'question_458', 'question_470', 'question_477', 'question_484', 'question_21', 'question_33', 'question_40', 'question_72', 'question_100', 'question_121', 'question_127', 'question_140', 'question_152', 'question_156', 'question_160', 'question_164', 'question_165', 'question_173', 'question_184', 'question_188', 'question_204', 'question_216', 'question_219', 'question_256', 'question_278', 'question_281', 'question_282', 'question_288', 'question_301', 'question_319', 'question_326', 'question_337', 'question_350', 'question_356', 'question_365', 'question_367', 'question_400', 'question_401', 'question_402', 'question_408', 'question_428', 'question_449', 'question_460', 'question_474', 'question_486', 'question_488', 'question_493', 'question_499', 'question_68', 'question_99', 'question_137', 'question_148', 'question_155', 'question_162', 'question_170', 'question_177', 'question_192', 'question_207', 'question_210', 'question_212', 'question_231', 'question_275', 'question_300', 'question_327', 'question_332', 'question_335', 'question_354', 'question_386', 'question_395', 'question_405', 'question_420', 'question_472', 'question_473', 'question_496', 'question_58', 'question_97', 'question_209']

question_lookup = {}

if not os.path.exists(experiment_input):
    print(f"找不到原始結果檔案: {experiment_input}")
    exit()

with open(experiment_input, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        qid = entry.get("question_id")
        question = entry.get("question")
        if qid and question:
            question_lookup[qid] = question

# === 重新跑的目標問題 ===
target_questions = []

for qid in specified_question_id:
    if qid in question_lookup:
        target_questions.append((qid, question_lookup[qid]))
    else:
        print(f"⚠️ 找不到對應內容: {qid}")


# === 開始執行實驗 ===
for question_id, question in target_questions:
    print(f"\n--- 重新處理問題: {question_id} ---")

    cqsolver_gpt4o = CQ_Solver(
        llm="gpt-4o",
        system_prompt=CQ_SYSTEM_PROMPT,
        max_turns=9,
        debug_log="CQ_Solver_debug_2.log",
        summary_json="CQ_Solver_summary_2.json"
    )

    system_name = "CQ_Solver"
    model_name = "gpt-4o"

    try:
        response = cqsolver_gpt4o.run(question)
        print(f"  ✅ 回答: {response}")
        result = {
            "system": system_name,
            "model": model_name,
            "question_id": question_id,
            "question": question,
            "answer": response,
            "error": None
        }
    except Exception as e:
        error_message = str(e)
        print(f"  💥 發生錯誤: {error_message}")
        result = {
            "system": system_name,
            "model": model_name,
            "question_id": question_id,
            "question": question,
            "answer": None,
            "error": error_message
        }

    # === 即時寫入結果 ===
    with open(output_file, 'a', encoding='utf-8') as f_out:
        json.dump(result, f_out, ensure_ascii=False)
        f_out.write('\n')

print("\n✅ 重新處理指定問題已完成。結果已更新至 experiment_results_2.jsonl")


--- 重新處理問題: question_111 ---
  ✅ 回答: Depictions of Indians in popular culture have played a significant role in swaying public opinion through various means, such as reinforcing stereotypes, influencing policy, and evolving representations.

### Key Insights:

1. **Stereotypes in Popular Culture**:
   - Indians have often been portrayed through stereotypical lenses in popular culture. Common depictions include exoticism, overemphasis on poverty, generalization of Indian culture as monolithic, and exaggerated accents. These stereotypes simplify and misrepresent the diversity of Indian culture, leading to misconceptions about Indian societal norms and lifestyles. This oversimplification influences how audiences perceive Indians, often reinforcing negative or superficial understandings.

2. **Evolution of Depictions**:
   - Historically, depictions of Indians in media were influenced by colonial ideologies, often portraying them as 'noble savages' or exotically spiritual. Over time, espe

In [44]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

eval_prompt_p1 = """You are asked to assess the quality of an AI assistant's answer to a user's question as an impartial judge. Since the type of answer you are evaluating is [Solve Professional Problem], you need to evaluate the answer in the following 5 criteria:
1. Factuality: Whether the information provided is accurate and based on reliable facts and data.
2. User Satisfaction: Whether the response meets the user's question and needs and provides a comprehensive and appropriate answer to the question.
3. Clarity: Whether the response is clear and understandable, and whether it uses concise language and structure so that the user can easily understand it.
4. Logical Coherence: Whether the response maintains overall consistency and logical coherence between different sections, avoiding self-contradiction.
5. Completeness: Whether the response provides sufficient information and details to meet the user's needs, and whether it avoids omitting important aspects.
6. Note that a longer answer is not always better, the answer that is concise and meets the above requirements is the best.

We will provide you with the user's question, an 8-score reference answer, and answers from the AI assistant that needs your assessment. When starting your evaluation, you need to follow the reasoning steps below:
1. Compare the AI assistant's answer with the reference answer, point out any shortcomings in the AI assistant's answer, and explain further.
2. Evaluate the AI assistant's answer in terms of the different criteria, giving each criterion a score from 1 to 10 after the evaluation of each.
3. Finally, combine the evaluations from each criterion and give the AI assistant's answer a composite score of 1 to 10.
4. Your scoring needs to be as rigorous as possible and adhere to the following scoring rules: in general, the higher the quality of the model's answers, the higher the score.
The two most important criteria are factual correctness and fulfillment of user needs, and the scores for these two dimensions dominate the final composite score.

When the model answer has irrelevance to the question, or intrinsically factually incorrect, or generates harmful content, the total score should be 1 to 2;
When the model answer has no serious errors and is largely harmless, but is of low quality and does not meet user requirements, the total score must be 3 to 4;
When the model answer basically meets the user's needs but performs poorly on some criteria and is of medium quality, the total score can be 5 to 6;
When the quality of the model response is similar to the reference answer and performs well in all criteria, the total score should be 7 to 8;
A score of 9 to 10 can only be achieved if the model significantly exceeds the quality of the reference answer, adequately addresses the user's question and all the needs, and is close to a perfect score on all criteria. As an example, the reference answer would receive a score of 8.

You need to evaluate and explain before you score. Your explanation of each criterion needs to be followed by the scoring. After that, at the end of your answer, return all of your scores in the following dictionary format, including the curly brackets, and make sure that your scores are integers:
{'Dimension 1': scoring, 'Dimension 2': scoring, ... , 'Final Score': Score}, e.g. {'Factuality': 9, 'User Satisfaction': 6, ... , 'Final Score': 7}.
"""


def compare_answers(filename="experiment_results.jsonl"):
    """
    開啟 JSONL 檔案，針對相同的 question_id，比較不同系統的答案，並回傳問題內容。
    "MindSearch" (gpt-4o) 的答案被視為參考答案，
    "ReAct" (gpt-4o) 和 "CQ_Solver" (gpt-4o) 的答案將與之比較。

    Args:
        filename (str): JSONL 檔案的名稱。

    Returns:
        dict: 一個字典，key 是 question_id，value 是一個包含問題、參考答案和要比較答案的字典。
              例如:
              {
                  "question_498": {
                      "question": "how does case management improve patient outcomes",
                      "reference_answer": "Case management...",
                      "ReAct_answer": "...",
                      "CQ_Solver_answer": "..."
                  },
                  "question_499": {
                      "question": "...",
                      "reference_answer": "...",
                      "ReAct_answer": "...",
                      "CQ_Solver_answer": "..."
                  },
                  ...
              }
    """
    results = {}
    data_by_question_id = {}

    with open(filename, 'r') as f:
        for line in f:
            try:
                record = json.loads(line.strip())
                question_id = record.get("question_id")
                question = record.get("question")
                system = record.get("system")
                model = record.get("model")
                answer = record.get("answer")

                if question_id and model == "gpt-4o":
                    if question_id not in data_by_question_id:
                        data_by_question_id[question_id] = {
                            "question": question}
                    data_by_question_id[question_id][system] = answer
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON line: {line.strip()} - {e}")
                continue

    for question_id, data in data_by_question_id.items():
        if "MindSearch" in data:
            comparison_data = {"question": data.get(
                "question"), "reference_answer": data["MindSearch"]}
            if "ReAct" in data:
                comparison_data["ReAct_answer"] = data["ReAct"]
            if "CQ_Solver" in data:
                comparison_data["CQ_Solver_answer"] = data["CQ_Solver"]
            results[question_id] = comparison_data

    return results



In [46]:
import json
import os
from openai import OpenAI

def compare_answers_for_batch(filename="experiment_results.jsonl", output_batch_file="batch_input.jsonl"):
    """
    開啟 JSONL 檔案，針對相同的 question_id，準備 Batch API 的輸入檔案。
    """
    batch_requests = []
    data_by_question_id = {}

    with open(filename, 'r') as f:
        for line in f:
            try:
                record = json.loads(line.strip())
                question_id = record.get("question_id")
                question = record.get("question")
                system = record.get("system")
                model = record.get("model")
                answer = record.get("answer")

                if question_id and model == "gpt-4o":
                    if question_id not in data_by_question_id:
                        data_by_question_id[question_id] = {"question": question}
                    data_by_question_id[question_id][system] = answer
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON line: {line.strip()} - {e}")
                continue

    for i, (question_id, data) in enumerate(data_by_question_id.items()):
        question = data.get("question", "N/A")
        reference_answer = data.get("MindSearch", "N/A")
        # react_answer = data.get("ReAct", "N/A") 
        cq_solver_answer = data.get("CQ_Solver", "N/A")

        # 針對 ReAct 的答案
#         if react_answer != "N/A":
#             eval_prompt_p2_react = f"""Question: "{question}"
# <Reference Answer>
# {reference_answer}
# </Reference Answer>

# <AI assistant's answer>
# {react_answer}
# </AI assistant's answer>"""
#             batch_requests.append({
#                 "custom_id": f"{question_id}-react",
#                 "method": "POST",
#                 "url": "/v1/chat/completions",
#                 "body": {
#                     "model": "gpt-4o",
#                     "messages": [{"role": "developer", "content": eval_prompt_p1},
#                                  {"role": "user", "content": eval_prompt_p2_react}]
#                 }
#             })

        # 針對 CQ_Solver 的答案
        if cq_solver_answer != "N/A":
            eval_prompt_p2_cq_solver = f"""Question: "{question}"
<Reference Answer>
{cq_solver_answer}
</Reference Answer>

<AI assistant's answer>
{reference_answer}
</AI assistant's answer>"""
            batch_requests.append({
                "custom_id": f"{question_id}-cq-solver",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o",
                    "messages": [{"role": "developer", "content": eval_prompt_p1},
                                 {"role": "user", "content": eval_prompt_p2_cq_solver}]
                }
            })

    # 將請求寫入 Batch 輸入檔案
    with open(output_batch_file, 'w') as f:
        for req in batch_requests:
            f.write(json.dumps(req) + '\n')

if __name__ == "__main__":
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    output_batch_file = "batch_input_exchange.jsonl"
    # compare_answers_for_batch(filename="merged_experiment_results.jsonl", output_batch_file=output_batch_file)
    # print(f"Batch input file created: {output_batch_file}")

    # 步驟 2: 上傳 Batch 輸入檔案
    try:
        with open(output_batch_file, "rb") as f:
            batch_input_file = client.files.create(
                file=f,
                purpose="batch"
            )
        print(f"Batch input file uploaded with ID: {batch_input_file.id}")
        input_file_id = batch_input_file.id

        # 步驟 3: 創建 Batch
        batch = client.batches.create(
            input_file_id=input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={"description": "Evaluation batch job"}
        )
        print(f"Batch created with ID: {batch.id}")

        # **注意：在這裡，Batch 已經創建並開始處理。您需要額外的程式碼來：**
        # 1. 定期檢查 Batch 的狀態 (使用 client.batches.retrieve(batch.id))
        # 2. 當 Batch 狀態為 "completed" 時，獲取輸出檔案 ID (batch.output_file_id)
        # 3. 使用 client.files.content(batch.output_file_id) 下載結果檔案
        # 4. 解析結果檔案 (通常也是 JSONL 格式) 並處理評估結果

    except Exception as e:
        print(f"Error during Batch API interaction: {e}")

Batch input file uploaded with ID: file-53SoReWdgfxp134H81WH1u
Batch created with ID: batch_67fc0167d5348190a70b07ed8e2200ab


In [34]:
import json
import os
from openai import OpenAI

def prepare_cq_solver_re_evaluation_batch(cq_solver_results_file="experiment_results_2.jsonl",
                                          mindsearch_results_file="experiment_results.jsonl",
                                          output_batch_file="batch_input_cq_reEval.jsonl"):
    """
    準備 Batch API 輸入檔案，以重新評估 experiment_results_2.jsonl 中的 CQ_Solver 答案，
    並使用 experiment_results.jsonl 中對應的 MindSearch 答案作為參考。
    """
    batch_requests = []
    mindsearch_answers = {}
    cq_solver_data = {}

    # 載入 MindSearch 答案作為參考
    with open(mindsearch_results_file, 'r') as f_ms:
        for line in f_ms:
            try:
                record = json.loads(line.strip())
                if record.get("system") == "MindSearch" and record.get("model") == "gpt-4o" and record.get("question_id"):
                    mindsearch_answers[record["question_id"]] = record.get("answer")
            except json.JSONDecodeError as e:
                print(f"Error decoding MindSearch results: {e}")
                continue

    # 載入 CQ_Solver 的答案和問題
    with open(cq_solver_results_file, 'r') as f_cq:
        for line in f_cq:
            try:
                record = json.loads(line.strip())
                if record.get("system") == "CQ_Solver" and record.get("model") == "gpt-4o" and record.get("question_id"):
                    cq_solver_data[record["question_id"]] = {
                        "question": record.get("question"),
                        "answer": record.get("answer")
                    }
            except json.JSONDecodeError as e:
                print(f"Error decoding CQ_Solver results: {e}")
                continue

    # 準備 Batch API 請求
    for question_id, cq_solver_info in cq_solver_data.items():
        question = cq_solver_info.get("question", "N/A")
        cq_solver_answer = cq_solver_info.get("answer", "N/A")
        reference_answer = mindsearch_answers.get(question_id, "N/A")

        if cq_solver_answer != "N/A" and reference_answer != "N/A":
            eval_prompt_p2_cq_solver = f"""Question: "{question}"
<Reference Answer>
{reference_answer}
</Reference Answer>

<AI assistant's answer>
{cq_solver_answer}
</AI assistant's answer>"""
            batch_requests.append({
                "custom_id": f"{question_id}-cq-solver-re-eval",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o",
                    "messages": [{"role": "developer", "content": eval_prompt_p1},
                                 {"role": "user", "content": eval_prompt_p2_cq_solver}]
                }
            })

    # 將請求寫入 Batch 輸入檔案
    with open(output_batch_file, 'w') as f:
        for req in batch_requests:
            f.write(json.dumps(req) + '\n')

if __name__ == "__main__":
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    output_batch_file = "batch_input_cq_reEval.jsonl"
    # prepare_cq_solver_re_evaluation_batch(output_batch_file=output_batch_file)
    # print(f"Batch input file created: {output_batch_file}")

    # 步驟 2: 上傳 Batch 輸入檔案
    try:
        with open(output_batch_file, "rb") as f:
            batch_input_file = client.files.create(
                file=f,
                purpose="batch"
            )
        print(f"Batch input file uploaded with ID: {batch_input_file.id}")
        input_file_id = batch_input_file.id

        # 步驟 3: 創建 Batch
        batch = client.batches.create(
            input_file_id=input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={"description": "Re-evaluation batch job for CQ_Solver"}
        )
        print(f"Batch created with ID: {batch.id}")

        # **後續步驟：輪詢狀態、檢索結果、解析和記錄結果**
        # 您需要實現這些步驟，就像之前的流程一樣。

    except Exception as e:
        print(f"Error during Batch API interaction: {e}")

Batch input file uploaded with ID: file-LhecDq7PPqxy36u3yVhkaw
Batch created with ID: batch_67fb9069240c81909722ac8e3e07c294


In [54]:
from openai import OpenAI
client = OpenAI()

batch = client.batches.retrieve("batch_67fc0167d5348190a70b07ed8e2200ab")
print(batch)

Batch(id='batch_67fc0167d5348190a70b07ed8e2200ab', completion_window='24h', created_at=1744568679, endpoint='/v1/chat/completions', input_file_id='file-53SoReWdgfxp134H81WH1u', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1744569730, error_file_id=None, errors=None, expired_at=None, expires_at=1744655079, failed_at=None, finalizing_at=1744569684, in_progress_at=1744568680, metadata={'description': 'Evaluation batch job'}, output_file_id='file-JMuur29q4cdNKJcFx6C3P3', request_counts=BatchRequestCounts(completed=500, failed=0, total=500))


In [55]:
from openai import OpenAI
client = OpenAI()

file_response = client.files.content("file-JMuur29q4cdNKJcFx6C3P3")
output_filename = "exchange_Eval.jsonl"
with open(output_filename, 'w') as outfile:
    outfile.write(file_response.text)

In [ ]:
if __name__ == "__main__":
    comparison_results = compare_answers()
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    output_filename = "evaluation_results.jsonl"

    test_question_id = "question_01"
    processed_count = 0

    with open(output_filename, 'a') as outfile:
        for question_id, data in comparison_results.items():
            if question_id == test_question_id:
                continue

            question = data.get("question", "N/A")
            reference_answer = data.get("reference_answer", "N/A")
            react_answer = data.get("ReAct_answer", "N/A")
            cq_solver_answer = data.get("CQ_Solver_answer", "N/A")

            # 針對 ReAct 的答案
            if react_answer != "N/A":
                eval_prompt_p2_react = f"""User Question: "{question}"
<Reference Answer>
{reference_answer}
</Reference Answer>

<AI assistant's answer>
{react_answer}
</AI assistant's answer>"""

                try:
                    completion_react = client.chat.completions.create(
                        model="gpt-4o",
                        messages=[{"role": "developer", "content": eval_prompt_p1},
                                  {"role": "user", "content": eval_prompt_p2_react}],
                    )
                    result_react = completion_react.choices[0].message.content
                    token_usage_react = completion_react.usage.model_dump() if completion_react.usage else None
                    output_data_react = {
                        "system": "ReAct",
                        "model": "gpt-4o",
                        "question_id": question_id,
                        "result": result_react,
                        "token_usage": token_usage_react
                    }
                    outfile.write(json.dumps(output_data_react) + '\n')
                    print(f"Processed {question_id} - ReAct")
                except Exception as e:
                    print(f"Error processing {question_id} - ReAct: {e}")

            # 針對 CQ_Solver 的答案
            if cq_solver_answer != "N/A":
                eval_prompt_p2_cq_solver = f"""Question: "{question}"
<Reference Answer>
{reference_answer}
</Reference Answer>

<AI assistant's answer>
{cq_solver_answer}
</AI assistant's answer>"""

                try:
                    completion_cq_solver = client.chat.completions.create(
                        model="gpt-4o",
                        messages=[{"role": "developer", "content": eval_prompt_p1},
                                  {"role": "user", "content": eval_prompt_p2_cq_solver}],
                    )
                    result_cq_solver = completion_cq_solver.choices[0].message.content
                    token_usage_cq_solver = completion_cq_solver.usage.model_dump() if completion_cq_solver.usage else None
                    output_data_cq_solver = {
                        "system": "CQ_Solver",
                        "model": "gpt-4o",
                        "question_id": question_id,
                        "result": result_cq_solver,
                        "token_usage": token_usage_cq_solver
                    }
                    outfile.write(json.dumps(output_data_cq_solver) + '\n')
                    print(f"Processed {question_id} - CQ_Solver")
                except Exception as e:
                    print(f"Error processing {question_id} - CQ_Solver: {e}")

            processed_count += 1